<a href="https://colab.research.google.com/github/acho110/Projects-Resume/blob/main/ethicsproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("adult_reconstruction.csv")
df['low earner'] = np.where(df['income'] < 50000, 1, 0)

features = ['hours-per-week','age', 'workclass', 'education', 'relationship', 'race', 'gender', 'occupation']
X = df[features].copy()
X['groups'] = np.where(X['race'] == 'Asian-Pac-Islander', 1, 0)
y = df['low earner'].copy()
X = pd.get_dummies(X, columns = ['workclass', 'education', 'occupation', 'relationship'])


X.drop(['race', 'gender'], inplace = True, axis = 1)



In [ ]:
X

hours-per-week  age  groups  workclass_?  workclass_Federal-gov  \
0                  20   40       0            0                      0   
1                  40   21       0            0                      0   
2                  10   17       0            0                      0   
3                  50   51       0            0                      0   
4                  38   28       0            0                      0   
...               ...  ...     ...          ...                    ...   
49526              65   35       0            0                      0   
49527              77   37       0            0                      0   
49528              55   24       0            0                      0   
49529              40   24       0            0                      0   
49530              20   39       0            0                      0   

       workclass_Local-gov  workclass_Never-worked  workclass_Private  \
0                        0                       0                  1   
1                        0                       0                  1   
2                        0                       0                  1   
3                        0                       0                  1   
4                        0                       0                  1   
...                    ...                     ...                ...   
49526                    0                       0                  1   
49527                    0                       0                  0   
49528                    0                       0                  1   
49529                    0                       0                  1   
49530                    0                       0                  0   

       workclass_Self-emp-inc  workclass_Self-emp-not-inc  ...  \
0                           0                           0  ...   
1                           0                           0  ...   
2                           0                           0  ...   
3                           0                           0  ...   
4                           0                           0  ...   
...                       ...                         ...  ...   
49526                       0                           0  ...   
49527                       0                           1  ...   
49528                       0                           0  ...   
49529                       0                           0  ...   
49530                       0                           0  ...   

       occupation_Protective-serv  occupation_Sales  occupation_Tech-support  \
0                               0                 0                        1   
1                               0                 0                        0   
2                               0                 0                        0   
3                               0                 1                        0   
4                               0                 0                        0   
...                           ...               ...                      ...   
49526                           0                 0                        0   
49527                           0                 1                        0   
49528                           0                 1                        0   
49529                           0                 0                        0   
49530                           0                 0                        0   

       occupation_Transport-moving  relationship_Husband  \
0                                0                     0   
1                                0                     0   
2                                0                     0   
3                                0                     1   
4                                0                     0   
...                            ...                   ...   
49526                            0                     1   
49527  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12345)


clf = RandomForestClassifier(random_state = 1000)
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=1000)

In [ ]:
y_hat = clf.predict(X_test)


In [ ]:
def compute_metrics(y_hat,
                    y_test,
                    X_test):

    test = X_test.copy()
    test['outcome'] = y_test
    test['predicted'] = y_hat

    TP_0 = sum((y_hat == 1) & (y_test == 1) & (X_test['groups'] == 0))/sum((y_test == 1) & (X_test['groups'] == 0))
    FP_0 = sum((y_hat == 1) & (y_test == 0) & (X_test['groups'] == 0))/sum((y_test == 0) & (X_test['groups'] == 0))
    TN_0 = sum((y_hat == 0) & (y_test == 0) & (X_test['groups'] == 0))/sum((y_test == 0) & (X_test['groups'] == 0))
    FN_0 = sum((y_hat == 0) & (y_test == 1) & (X_test['groups'] == 0))/sum((y_test == 1) & (X_test['groups'] == 0))

    TP_1 = sum((y_hat == 1) & (y_test == 1) & (X_test['groups'] == 1))/sum((y_test == 1) & (X_test['groups'] == 1))
    FP_1 = sum((y_hat == 1) & (y_test == 0) & (X_test['groups'] == 1))/sum((y_test == 0) & (X_test['groups'] == 1))
    TN_1 = sum((y_hat == 0) & (y_test == 0) & (X_test['groups'] == 1))/sum((y_test == 0) & (X_test['groups'] == 1))
    FN_1 = sum((y_hat == 0) & (y_test == 1) & (X_test['groups'] == 1))/sum((y_test == 1) & (X_test['groups'] == 1))

    # Calculate measures for g0
    accuracy_g0 = accuracy_score(y_test, y_hat, sample_weight=X_test['groups'] == 0)
    precision_g0 = sum((y_test == 1) & (X_test['groups'] == 0) & (y_hat == 1))/sum((y_hat == 1) & (X_test['groups'] == 0))
    recall_g0 = TP_0/(TP_0 + FN_0)
    false_negative_rate_g0 = FN_0
    false_positive_rate_g0 = FP_0

    accuracy_g1 = accuracy_score(y_test, y_hat, sample_weight=X_test['groups'] == 1)
    precision_g1 = sum((y_test == 1) & (X_test['groups'] == 1) & (y_hat == 1))/sum((y_hat == 1) & (X_test['groups'] == 1))
    recall_g1 = TP_1/(TP_1 + FN_1)
    false_negative_rate_g1 = FN_1
    false_positive_rate_g1 = FP_1

    metrics_df = pd.DataFrame({
    "Group": ["0-OtherRaces", "0-OtherRaces", "0-OtherRaces", "0-OtherRaces", "0-OtherRaces",
              "1-Asian-Pac-Islander", "1-Asian-Pac-Islander", "1-Asian-Pac-Islander", "1-Asian-Pac-Islander", "1-Asian-Pac-Islander"],
    "Metric": ["Accuracy", "Precision", "Recall", "FNR","FPR", "Accuracy", "Precision", "Recall", "FNR","FPR"],
    "Value": [accuracy_g0, precision_g0, recall_g0, false_negative_rate_g0, false_positive_rate_g0,
              accuracy_g1, precision_g1, recall_g1, false_negative_rate_g1, false_positive_rate_g1]})

    return metrics_df


In [ ]:
metrics_df = compute_metrics(y_hat = y_hat, y_test = y_test, X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))
#It looks like Asian-Pacific-Islanders have a higher false negative rate. This means the algorithm is incorrectly
#predicting that Asian-Pacific-Islanders are higher earners when they are actually low earners. If we were to use
#this algorithm for decision making we would be choosing to not help Asian-Pac-Islanders who need help at a higher rate
#than other races.

Group      0-OtherRaces  1-Asian-Pac-Islander
Metric                                       
Accuracy       0.808886              0.799270
FNR            0.112556              0.121212
FPR            0.448889              0.407895
Precision      0.866437              0.848780
Recall         0.887444              0.878788


In [ ]:

group_counts = [sum(X_train['groups'] == 0), sum(X_train['groups'] == 1)]
most_represented_group_count = max(group_counts)
most_represented_group_count

38379

In [ ]:
X_train['weights'] = X_train['groups'].apply(lambda x: most_represented_group_count / group_counts[x])

In [ ]:
weights = X_train['weights']
X_train.drop(['weights'],axis=1,inplace=True)


clf_preprocess = RandomForestClassifier(random_state = 1000)
clf_preprocess.fit(X_train, y_train, weights)

#predictions on the testing set
y_hat_preprocess = clf_preprocess.predict(X_test)

metrics_df = compute_metrics(y_hat = y_hat_preprocess, y_test = y_test, X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

#This actually increased the gap in false negative rates


Group      0-OtherRaces  1-Asian-Pac-Islander
Metric                                       
Accuracy       0.812000              0.795620
FNR            0.109305              0.121212
FPR            0.446222              0.421053
Precision      0.867546              0.844660
Recall         0.890695              0.878788


In [ ]:
def split_train_and_test_by_group(X_train, X_test, y_train, y_test):


    train = X_train.copy()
    test = X_test.copy()
    train['outcome'] = y_train
    test['outcome'] = y_test

    # split train and test by group
    train_g1 = train[train['groups']==1]
    train_g0 = train[train['groups']==0]
    test_g1 = test[test['groups']==1]
    test_g0 = test[test['groups']==0]

    # separate outcomes from test
    y_train_g1 = train_g1['outcome']
    y_train_g0 = train_g0['outcome']
    y_test_g1 = test_g1['outcome']
    y_test_g0 = test_g0['outcome']

    X_train_g1 = train_g1.drop(['outcome', 'groups'], axis=1)
    X_train_g0 = train_g0.drop(['outcome', 'groups'], axis=1)
    X_test_g1 = test_g1.drop(['outcome', 'groups'], axis=1)
    X_test_g0 = test_g0.drop(['outcome', 'groups'], axis=1)

    return y_train_g1, y_train_g0, y_test_g1, y_test_g0, X_train_g1, X_train_g0, X_test_g1, X_test_g0


y_train_g1, y_train_g0, y_test_g1, y_test_g0, \
X_train_g1, X_train_g0, X_test_g1, X_test_g0 = split_train_and_test_by_group(X_train = X_train,
                                                                              X_test = X_test,
                                                                              y_train = y_train,
                                                                              y_test = y_test)





In [ ]:
clf_g0 = RandomForestClassifier(random_state=1000)
clf_g0.fit(X_train_g0, y_train_g0)
clf_g1 = RandomForestClassifier(random_state=1000)
clf_g1.fit(X_train_g1, y_train_g1)
y_hat_g0 = clf_g0.predict(X_test_g0)
y_hat_g1 = clf_g1.predict(X_test_g1)

In [ ]:
def merge_separate_models(y_hat_g1, y_hat_g0, y_test_g1, \
                          y_test_g0, X_test_g1, X_test_g0):

    test_g1 = X_test_g1.copy()
    test_g1['groups'] = 1
    test_g1['outcome'] = y_test_g1
    test_g1['pred'] = y_hat_g1

    test_g0 = X_test_g0.copy()
    test_g0['groups'] = 0
    test_g0['outcome'] = y_test_g0
    test_g0['pred'] = y_hat_g0

    # Merge back together
    test = pd.concat([test_g0, test_g1])

    y_test = test['outcome']
    y_hat = test['pred']
    X_test = test.drop(['outcome','pred'], axis=1)

    return y_test, y_hat, X_test


y_test_combined, \
y_hat_combined, \
X_test_combined = merge_separate_models(y_hat_g1 = y_hat_g1,
                                        y_hat_g0 = y_hat_g0,
                                        y_test_g1 = y_test_g1,
                                        y_test_g0 = y_test_g0,
                                        X_test_g1 = X_test_g1,
                                        X_test_g0 = X_test_g0)






In [ ]:
metrics_df = compute_metrics(y_hat = y_hat_combined,
                             y_test = y_test_combined,
                             X_test = X_test_combined)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

Group      0-OtherRaces  1-Asian-Pac-Islander
Metric                                       
Accuracy       0.811378              0.813869
FNR            0.111608              0.111111
FPR            0.441333              0.381579
Precision      0.868512              0.858537
Recall         0.888392              0.888889


In [ ]:
#change in false negative rate gap
initial_diff = .121212-.112556
new_diff = .111111-.111608
pct_change = 100 * ((new_diff - initial_diff)/initial_diff)
pct_change

-105.74168207024026

In [ ]:
threshold_values = [i/100 for i in range(50, 90)]
y_hat_probs = clf.predict_proba(X_test)[:,1]

In [ ]:
def evaluate_group_thresholds(y_true, y_proba, groups, threshold_values):

    unique_groups = np.unique(groups)
    # This enumerates all possible options for threshold values within the range specified
    all_combinations = list(product(threshold_values, repeat=len(unique_groups)))
    # Prepare a list to collect results
    results = []
    for combination in all_combinations:
        group_thresholds = {group: threshold for group, threshold in zip(unique_groups, combination)}
        #print(group_thresholds)
        # Apply group-specific thresholds to generate predictions
        y_pred = np.zeros(y_true.shape)
        for group, threshold in group_thresholds.items():
            group_mask = (groups == group)
            y_pred[group_mask] = (y_proba[group_mask] > threshold).astype(int)


        # Calculate FNR for each group
        fnrs = {}
        for group in unique_groups:
            group_mask = (groups == group)
            group_true = y_true[group_mask]
            group_pred = y_pred[group_mask]
            FN = sum((group_pred == 0) & (group_true == 1))
            TP = sum((group_pred == 1) & (group_true == 1))
            fnrs[group] = (FN)/(TP+FN)

        # Calculate overall accuracy
        overall_accuracy = accuracy_score(y_true, y_pred)

        # Calculate the FnR difference
        fnr_diff = fnrs[unique_groups[0]] - fnrs[unique_groups[1]]
        # Create a row of data as a list of four items (see columns for below for ordering)
        row = [combination[0], combination[1], overall_accuracy, fnr_diff]
        results.append(row)


    # Create a DataFrame from the collected results
    columns = ['Threshold 0', 'Threshold 1', 'Model Accuracy', 'FNR Difference']
    results_df = pd.DataFrame(results, columns=columns)

    return results_df


results = evaluate_group_thresholds(y_true = y_test,
                                    y_proba = y_hat_probs,
                                    groups = X_test['groups'],
                                    threshold_values = threshold_values)

In [ ]:
# Filter the DataFrame for rows where the absolute FR difference is less than 0.05.
fnr_threshold_subset = results[abs(results['FNR Difference']) < .0003]
print(fnr_threshold_subset)
# Find the index of the row with the maximum accuracy in the filtered subset
max_accuracy_index = fnr_threshold_subset['Model Accuracy'].idxmax()

# Retrieve the row with the maximum accuracy
max_accuracy_row = results.iloc[max_accuracy_index]
print(max_accuracy_row)

      Threshold 0  Threshold 1  Model Accuracy  FNR Difference
80           0.52         0.50        0.807510       -0.000123
81           0.52         0.51        0.807510       -0.000123
365          0.59         0.55        0.806400        0.000127
1180         0.79         0.70        0.766125       -0.000111
Threshold 0       0.520000
Threshold 1       0.500000
Model Accuracy    0.807510
FNR Difference   -0.000123
Name: 80, dtype: float64


In [ ]:
def generate_group_level_labels(y_proba, groups, max_accuracy_row):

    y_pred = np.zeros(y_proba.shape)
    for group in groups.unique():
        group_mask = (groups == group)
        j = 'Threshold 1'
        if group == 0:
            j = 'Threshold 0'
        y_pred[group_mask] = (y_proba[group_mask] > max_accuracy_row[j])

    return y_pred


y_hat_thresholds = generate_group_level_labels(y_proba = y_hat_probs,
                                               groups = X_test['groups'],
                                               max_accuracy_row = max_accuracy_row)


In [ ]:
metrics_df = compute_metrics(y_hat = y_hat_thresholds,
                             y_test = y_test,
                             X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

Group      0-OtherRaces  1-Asian-Pac-Islander
Metric                                       
Accuracy       0.807744              0.799270
FNR            0.121089              0.121212
FPR            0.425778              0.407895
Precision      0.871358              0.848780
Recall         0.878911              0.878788


In [ ]:
#so as it turns out the the decoupled classifier approach leads to the lowest false negative rate across the groups
#and does a good job minimizing the difference.
